In [ ]:
pip install dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score


#normalizing data
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv("/content/train.csv", error_bad_lines=False)
df.head(10)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income_>50K
0,67,Private,366425,Doctorate,16,Divorced,Exec-managerial,Not-in-family,White,Male,99999,0,60,United-States,1
1,17,Private,244602,12th,8,Never-married,Other-service,Own-child,White,Male,0,0,15,United-States,0
2,31,Private,174201,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
3,58,State-gov,110199,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,0
4,25,State-gov,149248,Some-college,10,Never-married,Other-service,Not-in-family,Black,Male,0,0,40,United-States,0
5,59,State-gov,105363,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,40,United-States,0
6,70,Private,216390,9th,5,Married-civ-spouse,Machine-op-inspct,Wife,White,Female,2653,0,40,United-States,0
7,35,Self-emp-not-inc,361888,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,60,Japan,0
8,28,Private,74784,HS-grad,9,Never-married,Handlers-cleaners,Not-in-family,White,Male,0,0,50,United-States,0
9,28,Private,118089,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,4386,0,45,United-States,1


In [3]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income_>50K'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43957 entries, 0 to 43956
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              43957 non-null  int64 
 1   workclass        41459 non-null  object
 2   fnlwgt           43957 non-null  int64 
 3   education        43957 non-null  object
 4   educational-num  43957 non-null  int64 
 5   marital-status   43957 non-null  object
 6   occupation       41451 non-null  object
 7   relationship     43957 non-null  object
 8   race             43957 non-null  object
 9   gender           43957 non-null  object
 10  capital-gain     43957 non-null  int64 
 11  capital-loss     43957 non-null  int64 
 12  hours-per-week   43957 non-null  int64 
 13  native-country   43194 non-null  object
 14  income_>50K      43957 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 5.0+ MB


In [5]:
df.dropna(inplace=True)

In [6]:
X = df.drop("income_>50K",axis=1)
y = df["income_>50K"]

In [7]:
df['income_>50K'].value_counts()

0    30635
1    10092
Name: income_>50K, dtype: int64

так как у нас дизбаланс классов, делаем resample

In [8]:
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X, y)

Разделим данные на train/test и сохраним тестовую выборку на диск

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros,
                                                    test_size=0.2, random_state=55)


In [10]:
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

Создадим необходимые пайплайны

In [11]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    

class NumberSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key, drop_first=True).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key, drop_first=True)
        test_columns = [col for col in X.columns]
        
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]
    
class CatNaNInputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        
        self.frequent_constant = 'None'

    def fit(self, X, y=None):
        self.frequent_constant = X.mode()[0]
        return self

    def transform(self, X):
        X = X.fillna(self.frequent_constant)
        return X

In [12]:
continuous_columns = X_train.select_dtypes(include='number').columns.to_list()
categorical_columns = X_train.select_dtypes(exclude='number').columns.to_list()

Соберем кусок, ответственный за feature engineering

In [13]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('nan_inputer', CatNaNInputer()),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                
                ('NAN', SimpleImputer(strategy='median')),
                
        
                
            ])
    
    final_transformers.append((cont_col, cont_transformer))
    
feats = FeatureUnion(final_transformers)

Добавим классификатор с подобранными параметрами

In [14]:
%%time
model = RandomForestClassifier(random_state=55, n_estimators=150,max_features=5)
pipeline = Pipeline([
    ('features', feats),
    ('classifier', model),
])

pipeline.fit(X_train, y_train)

CPU times: user 12.7 s, sys: 247 ms, total: 13 s
Wall time: 12.9 s


Сохраним модель (пайплайн)

In [15]:
with open("rf_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

Проверка работоспособности и качества пайплайна

In [16]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [17]:
X_test.head(5)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,59,Private,109015,Some-college,10,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,38,United-States
1,41,Private,289551,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,7688,0,40,United-States
2,23,Private,428299,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,United-States
3,65,Self-emp-inc,103824,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
4,73,Self-emp-not-inc,110102,HS-grad,9,Widowed,Farming-fishing,Not-in-family,White,Male,0,1668,77,United-States


In [18]:
with open('rf_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [19]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('workclass',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='workclass')),
                                                                 ('nan_inputer',
                                                                  CatNaNInputer()),
                                                                 ('ohe',
                                                                  OHEEncoder(key='workclass'))])),
                                                ('education',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='education')),
                                                                 ('nan_inputer',
                                                                 

In [20]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [21]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.6466666666666666, F-Score=0.942, Precision=0.925, Recall=0.960


Создаем сервис для обработки запросов к модели

In [22]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from flask import Flask, request, jsonify
import pandas as pd

In [24]:
from flask_ngrok import run_with_ngrok

In [26]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2Ahf1VHsChQ5XQnuBwThE4uQAkP_9Qy62Je4zTNVSXyuQAJX
!./ngrok http 80

--2022-06-17 12:10:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.1’

ngrok-stable-linux- 100%[===================>]  13.13M  18.0MB/s    in 0.7s    

2022-06-17 12:10:41 (18.0 MB/s) - ‘ngrok-stable-linux-amd64.tgz.1’ saved [13770165/13770165]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [27]:
# Загружаем обученные модели
with open('rf_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [28]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Запустить сервис и не глушить его, пока работаем

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    (age, workclass, fnlwgt, education, educational_num,
       marital_status, occupation, relationship, race, gender,
       capital_gain, capital_loss, hours_per_week, native_country) = [np.nan]*14

    request_json = request.get_json()
    
    if request_json["age"]:
        home_ownership = request_json['age']
    
    if request_json["workclass"]:
        annual_income = request_json['workclass']
                
    if request_json["fnlwgt"]:
        years_job = request_json['fnlwgt']

    if request_json["education"]:
        tax_liens = request_json['education']
    
    if request_json["educational-num"]:
        accounts = request_json['educational-num']
                
    if request_json["marital-status"]:
        years_history = request_json['marital-status']

    if request_json["occupation"]:
        max_credit = request_json['occupation']
    
    if request_json["relationship"]:
        n_problems = request_json['relationship']
                
    if request_json["race"]:
        last_delinquent = request_json['race']

    if request_json["gender"]:
        bankruptcies = request_json['gender']
    
    if request_json["capital-gain"]:
        purpose = request_json['capital-gain']
                
    if request_json["capital-loss"]:
        term = request_json['capital-loss']

    if request_json["hours-per-week"]:
        current_loan_amount = request_json['hours-per-week']
    
    if request_json["native-country"]:
        current_credit_balance = request_json['native-country']
                

    
     
    preds = model.predict_proba(pd.DataFrame({'age' : [age],
                                              'workclass' : [workclass],
                                              'fnlwgt' : [fnlwgt],
                                              'education' : [education],
                                              'educational-num' : [educational_num],
                                              'marital-status' : [marital_status],
                                              'occupation' : [occupation],
                                              'relationship' : [relationship],
                                              'race' : [race],
                                              'gender' : [gender],
                                              'capital-gain' : [capital_gain],
                                              'capital-loss' : [capital_loss],
                                              'hours-per-week' : [hours_per_week],
                                              'native-country' : [native_country]}))
      
   
    data["predictions"] = preds[:, 1][0]
    data["description"] = request_json

    data["success"] = True
    print('OK')


    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b86a-34-125-211-95.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [17/Jun/2022 12:11:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 12:11:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Jun/2022 12:13:05] "POST /predict HTTP/1.1" 200 -


OK


...

OK


127.0.0.1 - - [17/Jun/2022 12:20:26] "POST /predict HTTP/1.1" 200 -


OK


127.0.0.1 - - [17/Jun/2022 12:20:26] "POST /predict HTTP/1.1" 200 -


OK


Тестовый клиент

In [ ]:
(age, workclass, fnlwgt, education, educational_num,
       marital_status, occupation, relationship, race, gender,
       capital_gain, capital_loss, hours_per_week, native_country) = (58, 'State-gov',110199, 
                                                                    '7th-8th',4,'Married-civ-spouse','Transport-moving','Husband',
                                                                      'White','Male',0,0,40,'United-States')
body = {'age' : [age],
        'workclass' : [workclass],
        'fnlwgt' : [fnlwgt],
        'education' : [education],                               
            'educational-num' : [educational_num],
            'marital-status' : [marital_status],
            'occupation' : [occupation],
            'relationship' : [relationship],
            'race' : [race],
            'gender' : [gender],
            'capital-gain' : [capital_gain],
            'capital-loss' : [capital_loss],
            'hours-per-week' : [hours_per_week],
            'native-country' : [native_country]}                                     

In [ ]:
with app.test_client() as t:
    response = t.post('/predict', json=body)
    json_data = response.get_json()

json_data

OK


{'description': {'age': [58],
  'capital-gain': [0],
  'capital-loss': [0],
  'education': ['7th-8th'],
  'educational-num': [4],
  'fnlwgt': [110199],
  'gender': ['Male'],
  'hours-per-week': [40],
  'marital-status': ['Married-civ-spouse'],
  'native-country': ['United-States'],
  'occupation': ['Transport-moving'],
  'race': ['White'],
  'relationship': ['Husband'],
  'workclass': ['State-gov']},
 'predictions': 0.16666666666666666,
 'success': True}